# Huggingface Sagemaker-sdk extension example using `Trainer` class

## Installs requirements if you haven´t already done it and sets up ipywidgets for datasets in sagemaker studio

In [10]:
%%capture
!pip install -r ../requirements.txt --upgrade

In [8]:
%%capture
import os 
import IPython
if 'SAGEMAKER_TRAINING_MODULE' in os.environ:
    !conda install -c conda-forge ipywidgets -y
    IPython.Application.instance().kernel.do_shutdown(True) # has to restart kernel so changes are used

## Initializing Sagemaker Session with local AWS Profile

From outside these notebooks, `get_execution_role()` will return an exception because it does not know what is the role name that SageMaker requires.

To solve this issue, pass the IAM role name instead of using `get_execution_role()`.

Therefore you have to create an IAM-Role with correct permission for sagemaker to start training jobs and download files from s3. Beware that you need s3 permission on bucket-level `"arn:aws:s3:::sagemaker-*"` and on object-level     `"arn:aws:s3:::sagemaker-*/*"`. 

You can read [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) how to create a role with right permissions.

In [14]:
# local aws profile configured in ~/.aws/credentials
local_profile_name='hf-sm' # optional if you only have default configured

# role name for sagemaker -> needs the described permissions from above
role_name = "SageMakerRole"

In [16]:
import sagemaker
import os
try:
    sess = sagemaker.Session()
    role = sagemaker.get_execution_role()
except Exception:
    import boto3
    # creates a boto3 session using the local profile we defined
    if local_profile_name:
        os.environ['AWS_PROFILE'] = local_profile_name # setting env var bc local-mode cannot use boto3 session
        #bt3 = boto3.session.Session(profile_name=local_profile_name)
        #iam = bt3.client('iam')
        # create sagemaker session with boto3 session
        #sess = sagemaker.Session(boto_session=bt3)
    iam = boto3.client('iam')
    sess = sagemaker.Session()
    # get role arn
    role = iam.get_role(RoleName=role_name)['Role']['Arn']
    


print(role)


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


arn:aws:iam::558105141721:role/SageMakerRole


### Sagemaker Session prints

In [11]:
print(sess.list_s3_files(sess.default_bucket(),'datasets/')) # list objects in s3 under datsets/
print(sess.default_bucket()) # s3 bucketname
print(sess.boto_region_name) # aws region of sagemaker session

['datasets/imdb/small/test/dataset.arrow', 'datasets/imdb/small/test/dataset_info.json', 'datasets/imdb/small/test/state.json', 'datasets/imdb/small/test/test_dataset.pt', 'datasets/imdb/small/train/dataset.arrow', 'datasets/imdb/small/train/dataset_info.json', 'datasets/imdb/small/train/state.json', 'datasets/imdb/small/training/train_dataset.pt', 'datasets/imdb/test/dataset.arrow', 'datasets/imdb/test/dataset_info.json', 'datasets/imdb/test/state.json', 'datasets/imdb/train/dataset.arrow', 'datasets/imdb/train/dataset_info.json', 'datasets/imdb/train/state.json']
sagemaker-eu-central-1-558105141721
eu-central-1


# Imports

Since we are using the `.py` module directly from `huggingface/` we have to adjust our `sys.path` to be able to import our estimator

In [28]:
import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# Preprocessing the data

In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [14]:
# load dataset
dataset = load_dataset('imdb')

# download tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

#helper tokenizer function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])
test_dataset = test_dataset.shuffle().select(range(10000)) # smaller the size for test dataset to 10k 

# sample a to small dataset for training
#train_dataset = train_dataset.shuffle().select(range(2000)) # smaller the size for test dataset to 10k 
#test_dataset = test_dataset.shuffle().select(range(150)) # smaller the size for test dataset to 10k 


# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

# set format for pytorch
train_dataset.rename_column_("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.rename_column_("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

Reusing dataset imdb (/Users/philippschmid/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Reusing dataset imdb (/Users/philippschmid/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Loading cached shuffled indices for dataset at /Users/philippschmid/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-f7ed38da5ada7a37.arrow
Loading cached processed dataset at /Users/philippschmid/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-700f1c95c213ac49.arrow
Loading cached processed dataset at /Users/philippschmid/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-0cc657b0f33fcf97.arrow


## Upload data to sagemaker S3

In [15]:
import glob
def upload_data_to_s3(dataset=None,prefix='datasets',split_type='train'):
    """helper function with saves the dataset locally using dataset.save_to_disk() and upload its then to s3. """
    
    temp_prefix =f"{prefix}/{split_type}"
    # saves datasets in local directory
    dataset.save_to_disk(f"./{temp_prefix}")
    
    # loops over saved files and uploads them to s3 
    for file in glob.glob(f"./{temp_prefix}/*"):
        sess.upload_data(file, key_prefix=temp_prefix)

    # return s3 url to files for estimator.fit()
    return f"s3://{sess.default_bucket()}/{temp_prefix}"

In [16]:
prefix = 'datasets/imdb'

training_input_path  = upload_data_to_s3(dataset=train_dataset,prefix=prefix,split_type='train')
test_input_path      = upload_data_to_s3(dataset=test_dataset,prefix=prefix,split_type='test')

print(training_input_path)
print(test_input_path)


s3://sagemaker-eu-central-1-558105141721/datasets/imdb/train
s3://sagemaker-eu-central-1-558105141721/datasets/imdb/test


# [Sagemaker Experiment for tracking](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html)

Amazon SageMaker Experiments is a capability of Amazon SageMaker that lets you organize, track, compare, and evaluate your machine learning experiments.

SageMaker Experiments comes with its own Experiments SDK which makes the analytics capabilities easily accessible in Amazon SageMaker Notebooks. Because SageMaker Experiments enables tracking of all the steps and artifacts that went into creating a model, you can quickly revisit the origins of a model when you are troubleshooting issues in production, or auditing your models for compliance verifications.


[Sagemaker Experiment docs](https://sagemaker-experiments.readthedocs.io/en/latest/)

Sagemaker Experiment Components are:
* Experiment
* Trial
* Trial Component
* Tracker 


**Experiment:**  
An Amazon SageMaker experiment is a collection of trials.

**Trial**  
An execution of a data-science workflow with an experiment.
Consists of a list of trial component objects, which document individual activities within the workflow. An Trial_Component for example is a `TrainingJob`. If you run an `estimator.fit()` with a trial. The `estimator` will automatically log all hyperparameters and metrics to trial. 

**Trial Component:**  

A trial component is a stage in a trial.
Trial components are created automatically within the SageMaker runtime and may not be created directly. To automatically associate trial components with a trial and experiment supply an experiment config when creating a job. For example: https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html

**Tracker:**
A tracker records experiment information to a SageMaker trial component. (custom logger in `train.py`)
When creating a tracker within a SageMaker training or processing job, use the load method with no arguments to track artifacts to the trial component automatically created for your job. When tracking within a Jupyternotebook running in SageMaker, use the create method to automatically create a new trial component.

Trackers are Python context managers and you can use them using the Python with keyword. Exceptions thrown within the with block will cause the tracker’s trial component to be marked as failed. Start and end times are automatically set when using the with statement and the trial component is saved to SageMaker at the end of the block.






## [Create an Amazon SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-create.html)

AWS provides an extra sdk to use `sagemaker-experiments`. If you haven´t already installed go a head an run. 

```python
pip install sagemaker-experiments
````


In [17]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

#boto3
import boto3
import os

**create sagemaker session with correct aws profile**

In [22]:
if local_profile_name:
    os.environ['AWS_PROFILE'] = local_profile_name # setting env var bc local-mode cannot use boto3 session
    
ex_sm_sess = boto3.client('sagemaker')
sess_experiment = Experiment(sagemaker_boto_client=ex_sm_sess)

**create experiment and trial**

In [23]:
experiment = sess_experiment.create(experiment_name='hf-1') # ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,119}
trial = experiment.create_trial(trial_name='text-classification') # ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,119}

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


**list experiments**

In [24]:
for exp in Experiment.list():
    print(exp)
for trial in experiment.list_trials():
    print(trial)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


ExperimentSummary(experiment_name='hf-1',experiment_arn='arn:aws:sagemaker:eu-central-1:558105141721:experiment/hf-1',display_name='hf-1',creation_time=datetime.datetime(2020, 12, 28, 17, 44, 43, 808000, tzinfo=tzlocal()),last_modified_time=datetime.datetime(2020, 12, 28, 17, 44, 43, 867000, tzinfo=tzlocal()))
TrialSummary(trial_name='text-classification',trial_arn='arn:aws:sagemaker:eu-central-1:558105141721:experiment-trial/text-classification',display_name='text-classification',creation_time=datetime.datetime(2020, 12, 28, 17, 44, 43, 867000, tzinfo=tzlocal()),last_modified_time=datetime.datetime(2020, 12, 28, 17, 44, 43, 867000, tzinfo=tzlocal()))


**delete experiment**

In [86]:
#experiment.delete_all(action="--force")

## Importing custom sdk-extension for HuggingFace

In [34]:
training_input_path  = 's3://sagemaker-eu-central-1-558105141721/datasets/imdb/small/train'
test_input_path      = 's3://sagemaker-eu-central-1-558105141721/datasets/imdb/small/test'

In [26]:
%load_ext autoreload
%autoreload 2

In [27]:
from huggingface.estimator import HuggingFace

ModuleNotFoundError: No module named 'huggingface'

# Create an Estimator with an Experiment

[Metric Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html)

To find a metric, SageMaker searches the logs that your algorithm emits and finds logs that match the regular expression that you specify for that metric. 

Defining Training Metrics (SageMaker Python SDK)
Define the metrics that you want to send to CloudWatch by specifying a list of metric names and regular expressions as the metric_definitions argument when you initialize an Estimator object. For example, if you want to monitor both the train:error and validation:error metrics in CloudWatch, your Estimator initialization would look like the following:
```python
Estimator(
    ...,
    sagemaker_session = sm_sess,
    tags = [{'Key': 'my-experiments', 'Value': 'demo2'}])

estimator.fit(
    ...,
    experiment_config = {
        # "ExperimentName"
        "TrialName" : demo_trial.trial_name,
        "TrialComponentDisplayName" : "TrainingJob",
    })
```

In the regex for the train:error metric defined above, the first part of the regex finds the exact text `"Train_error="`, and the expression `(.*?);` captures zero or more of any character until the first `;` semicolon character.

For more information about training by using Amazon SageMaker Python SDK estimators, see https://github.com/aws/sagemaker-python-sdk#sagemaker-python-sdk-overview.

In [32]:
from huggingface.estimator import HuggingFace


huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='../scripts',
                            sagemaker_session=sess,
                            use_spot_instances=True,
                            max_wait=4600, # This should be equal to or greater than max_run in seconds'
                            max_run=3600,
                            base_job_name='huggingface-sdk-extension',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            framework_version={'transformers':'4.1.1','datasets':'1.1.3'},
                            py_version='py3',
                            hyperparameters = {'epochs': 3,
                                               'train_batch_size': 16,
                                               'model_name':'distilbert-base-uncased'})

In [33]:
trial.trial_name

'text-classification'

In [36]:
huggingface_estimator.image_uri

'558105141721.dkr.ecr.eu-central-1.amazonaws.com/huggingface-training:0.0.1-gpu-transformers4.1.1-datasets1.1.3-cu110'

In [37]:
huggingface_estimator.fit(
    {'train': training_input_path, 'test': test_input_path},
    experiment_config = {
        # "ExperimentName"
        "TrialName" : trial.trial_name,
        "TrialComponentDisplayName" : "TrainingJob",
    })

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: huggingface-sdk-extension-2020-12-28-16-53-59-249


2020-12-28 16:53:59 Starting - Starting the training job...
2020-12-28 16:54:23 Starting - Launching requested ML instancesProfilerReport-1609174439: InProgress
......
2020-12-28 16:55:25 Starting - Preparing the instances for training......
2020-12-28 16:56:31 Downloading - Downloading input data
2020-12-28 16:56:31 Training - Downloading the training image.....................
2020-12-28 16:59:55 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-28 16:59:56,073 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-12-28 16:59:56,097 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-28 16:59:57,522 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-12-28 16:59:57,992 sagemaker-training-toolkit INFO     Invoking user script

Training

{'eval_loss': 0.6602308750152588, 'eval_accuracy': 0.68, 'eval_f1': 0.44186046511627913, 'eval_precision': 0.95, 'eval_recall': 0.2878787878787879, 'epoch': 1.0}
{'eval_loss': 0.26919224858283997, 'eval_accuracy': 0.9066666666666666, 'eval_f1': 0.8955223880597014, 'eval_precision': 0.8823529411764706, 'eval_recall': 0.9090909090909091, 'epoch': 2.0}
{'eval_loss': 0.3070515990257263, 'eval_accuracy': 0.88, 'eval_f1': 0.8524590163934426, 'eval_precision': 0.9285714285714286, 'eval_recall': 0.7878787878787878, 'epoch': 3.0}
{'epoch': 3.0}
***** Eval results *****
#015Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]#015Downloading: 100%|██████████| 442/442 [00:00<00:00, 425kB/s]
#015Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]#015Downloading:   2%|▏         | 4.65M/268M [00:00<00:05, 46.5MB/s]#015Downloading:   4%|▎         | 9.91M/268M [00:00<00:05, 48.2MB/s]#015Downloading:   6%|▌         | 15.7M/268M [00:00<00:04, 50.8MB/s]#015Downloading:   8%|▊         | 21.7M/268M [0


2020-12-28 17:01:57 Uploading - Uploading generated training model
2020-12-28 17:02:35 Completed - Training job completed
Training seconds: 371
Billable seconds: 111
Managed Spot Training savings: 70.1%


## Fine-tune model with differen Hyperparamter and track them

in this example we are going to train 2 different estimator with different transformers models

In [81]:
training_input_path  = 's3://sagemaker-eu-central-1-558105141721/datasets/imdb/small/train'
test_input_path      = 's3://sagemaker-eu-central-1-558105141721/datasets/imdb/small/test'

In [87]:
model_experiment = sess_experiment.create(experiment_name='different-model-experiment') # ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,119}

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [88]:
learning_rate=[5e-5,1e-4]


In [ ]:
from smexperiments.trial import Trial
import time

for i, rate in enumerate(learning_rate):
    # defines trial_name in given input
    trial_name = f"hf-training-job-learning-rate-{i}-{int(time.time())}"
    
    # creates trial for our experiment
    hf_trial = model_experiment.create_trial(trial_name=trial_name)
    
    # creates HuggingFace Estimator
    hf_estimator = HuggingFace(entry_point='train.py',
                            source_dir='../scripts',
                            sagemaker_session=sess,
                            use_spot_instances=True,
                            max_wait=4600, # This should be equal to or greater than max_run in seconds'
                            max_run=3600,
                            base_job_name='huggingface-sdk-extension',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            framework_version={'transformers':'4.1.1','datasets':'1.1.3'},
                            py_version='py3',
                            metric_definitions=[
                                 {'Name': 'validation:eval_loss', 'Regex':"'eval_loss': (.*?),"},
                                 {'Name': 'validation:eval_accuracy', 'Regex': "'eval_accuracy': (.*?),"},
                                 {'Name': 'validation:eval_f1', 'Regex': "'eval_f1': (.*?),"},
                                 {'Name': 'validation:eval_precision', 'Regex': "'eval_precision': (.*?),"},
                                 {'Name': 'validation:eval_recall', 'Regex': "'eval_recall': (.*?),"}],
                            hyperparameters = {'epochs': 3,
                                               'train_batch_size': 8,
                                               'eval_batch_size': 16,
                                               'model_name': 'distilbert-base-uncased',
                                               'learning_rate': rate},
                            enable_sagemaker_metrics=True)

    # trains the model
    hf_estimator.fit(
        {'train': training_input_path, 'test': test_input_path}, 
        job_name=trial_name,
        experiment_config={
            "TrialName": hf_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
    )

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: hf-training-job-learning-rate-0-1609235016


2020-12-29 09:43:37 Starting - Starting the training job...
2020-12-29 09:44:01 Starting - Launching requested ML instancesProfilerReport-1609235017: InProgress
......
2020-12-29 09:45:02 Starting - Preparing the instances for training.........
2020-12-29 09:46:23 Downloading - Downloading input data
2020-12-29 09:46:23 Training - Downloading the training image.....................
2020-12-29 09:50:06 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-29 09:50:02,393 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-12-29 09:50:02,417 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-29 09:50:05,453 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-12-29 09:50:05,862 sagemaker-training-toolkit INFO     Invoking user script

Train

{'eval_loss': 0.6338126063346863, 'eval_accuracy': 0.7933333333333333, 'eval_f1': 0.7207207207207208, 'eval_precision': 0.8888888888888888, 'eval_recall': 0.6060606060606061, 'epoch': 1.0}
{'eval_loss': 0.2875109314918518, 'eval_accuracy': 0.9, 'eval_f1': 0.8920863309352518, 'eval_precision': 0.8493150684931506, 'eval_recall': 0.9393939393939394, 'epoch': 2.0}
{'eval_loss': 0.28652578592300415, 'eval_accuracy': 0.8733333333333333, 'eval_f1': 0.8503937007874016, 'eval_precision': 0.8852459016393442, 'eval_recall': 0.8181818181818182, 'epoch': 3.0}
{'epoch': 3.0}
***** Eval results *****
#015Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]#015Downloading: 100%|██████████| 442/442 [00:00<00:00, 460kB/s]
#015Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]#015Downloading:   2%|▏         | 4.13M/268M [00:00<00:06, 41.3MB/s]#015Downloading:   4%|▎         | 9.72M/268M [00:00<00:05, 44.8MB/s]#015Downloading:   6%|▌         | 15.5M/268M [00:00<00:05, 48.1MB/s]#015Downloading:   8%


2020-12-29 09:52:07 Uploading - Uploading generated training model
2020-12-29 09:52:48 Completed - Training job completed


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.


Training seconds: 391
Billable seconds: 117
Managed Spot Training savings: 70.1%


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: hf-training-job-learning-rate-1-1609235579


2020-12-29 09:53:00 Starting - Starting the training job...
2020-12-29 09:53:23 Starting - Launching requested ML instancesProfilerReport-1609235579: InProgress
.........
2020-12-29 09:54:44 Starting - Preparing the instances for training......
2020-12-29 09:56:04 Downloading - Downloading input data
2020-12-29 09:56:04 Training - Training in-progress...
2020-12-29 09:56:25 Training - Downloading the training image......

# NOT WORKING:  Use Sagemaker Experiments without Sagemaker 

you can use Sagemaker Experiments without sagemaker if you use a `Tracker` to log your metrics, inputs, parameters into Sagemaker. 

In [53]:
from smexperiments.experiment import Experiment
from smexperiments.tracker import Tracker
from smexperiments.trial_component import TrialComponent

In [69]:
# create sagemaker experiment
man_experiment = sess_experiment.create(experiment_name='manual-experiment3') # ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,119}

# create trial for experiment
man_trial = man_experiment.create_trial(trial_name='manuall-trial-2')

# creates tracker
man_tracker = Tracker.create()

# log hyperparameter of learning rate
man_tracker.log_parameter('learning_rate', 0.01)

# logs metrics
man_tracker.log_metric(metric_name='accuracy', value=0.9, iteration_number=1)
man_tracker.log_metric(metric_name='f1', value=0.7, iteration_number=1)


# add tracked values to trial
man_trial.add_trial_component(man_tracker)


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


ClientError: An error occurred (ValidationException) when calling the CreateTrialComponent operation: Trial Component creation is currently restricted to the SageMaker runtime. Try supplying an experiment config when creating a job instead.